<h1>Quora Question Pairs.</h1>
<h1>Quick guide for making 0.10112 score.</h1>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
train = pd.read_csv('../input/train.csv').fillna(" ")
train = train.dropna(how="any").reset_index(drop=True)

In [ ]:
train.head()

Посмотрим сколько дубликатов:

In [ ]:
train.groupby("is_duplicate")['id'].count().plot.bar()

Для выделения признаков из предложений воспользуемся CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
Bag = CountVectorizer(max_df=0.999, min_df=50, max_features=300, 
                                      analyzer='char', ngram_range=(1,2), 
                                      binary=True, lowercase=True)

Финальный score зависит от max_features и ngram_range
-----------------------------------------------------

[Для получения score = 0.31675][1]  нужно использовать следующее параметры: 

 - max_features = 300000
 - min_df = 50
 - ngram_range = (1,10)

То есть, если задать max_features = 10^10, можно получить желаемое первое место и score = 0.10112, все зависит от мощности компьютера. 

  [1]: https://www.kaggle.com/selfishgene/shallow-benchmark-0-31675-lb?scriptVersionId=1117887

In [ ]:
Bag.fit(pd.concat((train.question1,train.question2)).unique())

In [ ]:
question1 = Bag.transform(train['question1'])
question2 = Bag.transform(train['question2'])

In [ ]:
X = -(question1 != question2).astype(int)
y = train['is_duplicate'].values

Следует заметить, что X принимает следующие значения:

 - 0, если конкретная последовательность букв присутствует в обоих вопросах.
 - -1, если конкретная последовательность букв присутствует в одном вопросе, но не присутствует в другом.

<center><h3>Применим логистическую регрессию</h3></center>

In [ ]:
logisticRegressor = linear_model.LogisticRegression(C=0.1, solver='sag', 
                                                    class_weight={1: 0.472008228977, 0: 1.30905513329})
logisticRegressor.fit(X, y)

Стоит заметить: class_weight принимает значения {0.472008228977,  1.30905513329} по следующим причинам. 


Оказывается, что распределение меток на трейне и тесте разное, то есть  распределение меток на тесте просто скошено.  А прогноз модели следует делать по следующей формуле: 

$$f(x) = \frac{\gamma_1 x}{\gamma_1 x + \gamma_0 (1-x)},$$

где $$\gamma_0 = 1.30905513329, \gamma_1 = 0.472008228977.$$ 

И график этой функции выглядит следующим образом: 

In [ ]:
gamma_0 = 1.30905513329
gamma_1 = 0.472008228977
def link_function(x):
    return gamma_1*x/(gamma_1*x + gamma_0*(1 - x))

support = np.linspace(0, 1, 1000)
values = link_function(support)

fig, ax = plt.subplots()
ax.plot(support, values)
ax.set_title('Link transformation', fontsize=20)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
plt.show()

<h3>Применяю к тестовой выборке</h3>

In [ ]:
test = pd.read_csv('../input/test.csv')
test.ix[test['question1'].isnull(),['question1','question2']] = ' '
test.ix[test['question2'].isnull(),['question1','question2']] = ' '

In [ ]:
%%time
test.ix[test['question1'].isnull(),['question1','question2']] = ' '
test.ix[test['question2'].isnull(),['question1','question2']] = ' '

Question1 = Bag.transform(test['question1'])
Question2 = Bag.transform(test['question2'])

X_test = -(Question1 != Question2).astype(int)

seperators= [750000,1500000]
testPredictions1 = logisticRegressor.predict_proba(X_test[:seperators[0],:])[:,1]
testPredictions2 = logisticRegressor.predict_proba(X_test[seperators[0]:seperators[1],:])[:,1]
testPredictions3 = logisticRegressor.predict_proba(X_test[seperators[1]:,:])[:,1]
testPredictions = np.hstack((testPredictions1,testPredictions2,testPredictions3))

<center><h3>Делаем сабмит</h3></center>

In [ ]:
submissionName = 'quora_submission'

submission = pd.DataFrame()
submission['test_id'] = test['test_id']
submission['is_duplicate'] = testPredictions
submission.to_csv(submissionName + '.csv', index=False)

Итог: можно бесконечно увеличивать max_features, ограничение - мощность компьютера. Это достаточно простая модель, но она дает неплохие результаты.